# Libraries

In [1]:
import os

# go up one directory
os.chdir("..")

import pandas as pd 
import numpy as np
from tqdm import tqdm
from sklearn_extra.cluster import KMedoids
from sklearn.decomposition import PCA
from functions.cosmic_val import *
from functions.graph_tools import *
from functions.data_handling import data_augmentation
from models.muse import *
from functions import cosmic_val
from functions import data_handling as dh
from tqdm import tqdm
from skopt import gp_minimize
from skopt.space import Real
from collections import defaultdict


# # set seed
np.random.seed(123)
torch.manual_seed(123)

# Data

In [2]:
data_path = "data/catalogues_Ovary_SBS.tsv"
cosmic_path = "data/COSMIC_v3.4_SBS_GRCh37.txt"
output_folder = "data/processed"
output_filename = "Ordered_Ovary_SBS.csv"
ordered_data_path = os.path.join(output_folder, output_filename)

In [3]:
dh.load_preprocess_data(data_path, cosmic_path, sep1 = "\t", sep2 = "\t", output_folder = output_folder, output_filename = output_filename)

Data already exists in  data/processed/Ordered_Ovary_SBS.csv


In [4]:
# load data
data = pd.read_csv(ordered_data_path, index_col = 0)
cosmic = pd.read_csv(cosmic_path, sep = "\t", index_col = 0)

In [5]:
L_ONE = 128
TOLERANCE = 1e-10
CONSTRAINT = 'identity'

augmented_data = data_augmentation(X=data, augmentation=50)

In [6]:
print(np.shape(augmented_data))

(96, 26150)


In [ ]:
# Dictionary of lists: {k: [iterations]}
results_dict = defaultdict(list)

losses_train = []
signatures = []
iterations = 10
k_range = 10

augmented_data = data_augmentation(X=data, augmentation=50)

for k in tqdm(range(3, k_range)):
    for i in range(iterations):
        
        muse_model = HybridAutoencoder(input_dim=data.shape[0], 
                                        l_1=L_ONE,
                                        latent_dim=k,
                                        weights = 'xavier',)

        # Training MUSE
        muse_error, muse_signatures, muse_exposures, muse_train_loss, muse_val_loss = train_model_for_extraction(
            model=muse_model,
            X_aug_multi_scaled=augmented_data.T,
            X_scaled=data.T,
            signatures=k,
            epochs=1000,
            batch_size=64,
            save_to='muse_test',
            iteration=i,
            patience=15,
            beta = 0.01,
            lr = 0.001
        )

        # Normalize signatures
        diagonals_muse = muse_signatures.sum(axis=0)
        muse_exposures = muse_exposures.T @ np.diag(diagonals_muse)
        muse_signatures = muse_signatures @ np.diag(1 / diagonals_muse)

        # Store results
        losses_train.append(muse_train_loss)
        signatures.append(muse_signatures)

        # Store data in structured format
        results_dict[k].append({
            "iteration": i,
            "muse_error": muse_error, # This is the reconstruction error
            "muse_signatures": muse_signatures  # Keep as NumPy array for easier processing
        })

    # Convert dictionary into a DataFrame for better analysis
    df_results = pd.DataFrame([
    {"k": k, "iteration": entry["iteration"], "muse_error": entry["muse_error"], "muse_signatures": entry["muse_signatures"]}
    for k, entries in results_dict.items()
    for entry in entries
    ])


In [ ]:
k = find_best_k(df_results, avg_threshold= 0.5, min_threshold=0.2)

####  Optimize hyperparams & then run bunch of times. Adam lr & beta

In [ ]:
# Objective function to minimize (muse_error)
def objective(params):
    beta, lr = params
    
    muse_model = HybridAutoencoder(
        input_dim=data.shape[0],  # 96
        l_1=L_ONE,
        latent_dim=k,  # Best k found earlier
        weights='xavier',
    )

    # Train model with candidate beta and learning rate
    muse_error, _, _, _, _ = train_model_for_extraction(
        model=muse_model,
        X_aug_multi_scaled=augmented_data.T,
        X_scaled=data.T,
        signatures=k, # Best k found earlier
        epochs=1000,
        batch_size=64,
        save_to='muse_test',
        iteration=0,  # Single run per candidate
        patience=15,
        lr=lr,  # Optimized learning rate
        beta=beta  # Optimized beta
    )
    
    return muse_error  # Lower is better

# Define search space
search_space = [
    Real(1e-4, 1e-2, name="beta"), 
    Real(1e-4, 1e-2, name="lr")
]

# Run Bayesian Optimization
result = gp_minimize(objective, search_space, n_calls=20, random_state=123)

# Best hyperparameters found
best_beta, best_lr = result.x
print(f"Optimal beta: {best_beta}, Optimal lr: {best_lr}")

In [ ]:
# Run MUSE with optimized hyperparameters
results_dict = defaultdict(list)

iterations = 30


for i in range(iterations):
    muse_model = HybridAutoencoder(
        input_dim=data.shape[0],  # 96
        l_1=L_ONE,
        latent_dim=k,  # Best k found earlier
        weights='xavier',
    )

    muse_error, muse_signatures, muse_exposures, muse_train_loss, muse_val_loss = train_model_for_extraction(
        model=muse_model,
        X_aug_multi_scaled=augmented_data.T,
        X_scaled=data.T,
        signatures=k, # Best k found earlier
        epochs=2000,
        batch_size=64,
        save_to='muse_test',
        iteration=0,  # Single run
        patience=15,
        lr=best_lr,  # Optimized learning rate
        beta=best_beta  # Optimized beta
    )

    # Normalize signatures

    diagonals_muse = muse_signatures.sum(axis=0)
    muse_exposures = muse_exposures.T @ np.diag(diagonals_muse)
    muse_signatures = muse_signatures @ np.diag(1 / diagonals_muse)

    # Store results

    losses_train.append(muse_train_loss)
    signatures.append(muse_signatures)

    # Store data in structured format

    results_dict[k].append({
        "iteration": i,
        "muse_error": muse_error,  # This is the reconstruction error
        "muse_signatures": muse_signatures  # Keep as NumPy array for easier processing
    })

    # Convert dictionary into a DataFrame for better analysis

df_results = pd.DataFrame([
    {"k": k, "iteration": entry["iteration"], "muse_error": entry["muse_error"], "muse_signatures": entry["muse_signatures"]}
    for k, entries in results_dict.items()
    for entry in entries
])

In [ ]:
signatures = df_results[df_results['k'] == k]['muse_signatures'].values # Get signatures

all_signatures = np.hstack(signatures)

In [ ]:
pam = KMedoids(n_clusters = k, metric='cosine').fit(all_signatures.T)
labels = pam.labels_
medoid_indices = pam.medoid_indices_
consensus_signatures = all_signatures[:, medoid_indices]

In [ ]:
matched_signatures, mean_similarity = cosmic_val.compute_match(consensus_signatures, cosmic, index = 0)

In [ ]:
print(matched_signatures)
print("\nMean similarity of the matched signatures: ", mean_similarity)

In [ ]:
all_matches = cosmic_val.compute_all_matches(all_signatures, cosmic, k = k)

In [ ]:
all_matches

In [ ]:
plot_cosine_similarity_matrix(all_matches, title = "Cosine similarity matrix non-linear AE", figsize=(24,12), legend_colums = 4, palette = 'tab20')

In [ ]:
plot_signature_frequency(all_matches, title = "Signature frequency non-linear AE", figsize=(12,7), palette = 'tab20')

In [ ]:
reduced_signatures = PCA(n_components=2).fit_transform(all_signatures.T)
plot_clusters(reduced_signatures, labels, medoid_indices, k, "non-linear AE signature clusters")

In [ ]:
df_consensus = pd.DataFrame(consensus_signatures, index = data.index)

In [ ]:
plot_signature(df_consensus, "non-linear AE")